<a href="https://colab.research.google.com/github/David-Mihnea/git_test/blob/main/Route1_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project Title & Team Info

**Project Title**: _Workshop 1: WiDS University Datathon 2026_  
**Team Name**: _Research DUO_  
**University**: _Bucharest University of Economic Studies_  
**Course**: _Software Open Source for Statistics and Data Science_  
**Term**: _1st Semester, 2025_  

**Team Members**:  
- Gianina Petrașcu (GitHub: [@gianinapetrascu](https://github.com/gianinapetrascu))  
- Ioana Bîrlan (GitHub: [@ioanabirlan](https://github.com/ioanabirlan))  


### 🔹 Route 1: Accelerating Equitable Evacuations

**Core Question:**  
*How can we reduce delays in evacuation alerts and improve response times for the communities that are most at risk?*

This route focuses on analyzing how and when evacuation alerts are triggered — and how we can improve timeliness and fairness in communication, especially for vulnerable populations.

## Dataset Overview

Summarize the datasets you used and how you processed them.

- `infrastructure.csv`: Metadata and coordinates of infrastructure
- `fire_perimeters.geojson`: Timestamped fire perimeter polygons
- `evacuation_zones.csv`: (Optional) evacuation declarations
- `watch_duty_change_log.csv`: Alerts and timestamps
- (Optional) NOAA weather data or census data

**Load data**

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving wids-university-datathon-2025.zip to wids-university-datathon-2025.zip


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c wids-university-datathon-2025


In [ ]:
!unzip wids-university-datathon-2025.zip -d data/

In [ ]:
#libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_theme()
plt.colormaps()

from sklearn.model_selection import train_test_split


In [ ]:
events    = pd.read_csv("data/geo_events_geoevent.csv", low_memory=True)
changes   = pd.read_csv("data/geo_events_geoeventchangelog.csv", low_memory=True)
perims    = pd.read_csv("data/fire_perimeters_gis_fireperimeter.csv", low_memory=True)
evac      = pd.read_csv("data/evac_zones_gis_evaczone.csv", low_memory=True)
evac_map  = pd.read_csv("data/evac_zone_status_geo_event_map.csv", low_memory=True)

In [ ]:
print("Loaded:")
for name, df in zip(["events","changes","perims","evac","evac_map"],
                    [events,changes,perims,evac,evac_map]):
    print(f"  {name:10s} {df.shape}")

In [ ]:
events.head()

In [ ]:
changes.head()

In [ ]:
perims.head()

In [ ]:
evac.head()

In [ ]:
evac_map.head()

In [ ]:
import pandas as pd
import numpy as np
import json

events = events[[
    "id", "date_created", "date_modified", "geo_event_type",
    "name", "is_active", "lat", "lng", "data"]].copy()

def extract_json_field(js, key):
    if not isinstance(js, str) or "{" not in js:
        return None
    try:
        js = js.strip().strip('"').strip("'")
        parsed = json.loads(js)
        return parsed.get(key)
    except Exception:
        return None

for col in [
    "is_prescribed", "is_fps", "containment", "acreage",
    "evacuation_notes", "evacuation_orders", "evacuation_warnings"]:
    events[col] = events["data"].apply(lambda x: extract_json_field(x, col))

events.drop(columns=["data"], inplace=True, errors="ignore")

changes = changes[[c for c in ["geo_event_id","date_created","changes"] if c in changes.columns]].copy()

perims = perims[[c for c in [
    "geo_event_id","approval_status","date_created",
    "source_date_current","geom","source_acres"] if c in perims.columns]].copy()
if "approval_status" in perims.columns:
    perims = perims.query("approval_status == 'approved'").copy()

evac = evac[[c for c in [
    "uid_v2","display_name","region_id","is_active",
    "status","source_attribution","geom","geom_label"] if c in evac.columns]].copy()

assert {"geo_event_id","uid_v2"}.issubset(evac_map.columns), "Mapping table must include geo_event_id and uid_v2"

print("\nAfter cleaning:")
print(f"  events   {events.shape}")
print(f"  changes  {changes.shape}")
print(f"  perims   {perims.shape} (approved only)")
print(f"  evac     {evac.shape}")

print("\nExtracted fields preview:")
print(events[["id","geo_event_type","containment","acreage","is_fps","is_prescribed"]].head(10))

print("\nNon-null counts for extracted fields:")
print(events[["containment","acreage","is_fps","is_prescribed"]].notna().sum())


   - events: 62,696 rows -> wildfire or location incidents.
   - changes: 178,697 rows -> logs of event updates (timing & progression).
   - perims: 4,139 rows (approved only) -> valid fire perimeters.
   - evac: 37,458 rows -> evacuation zone data.

In [ ]:
# timestamps sparsing

def to_dt(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce", utc=True)
    return df

events  = to_dt(events,  ["date_created","date_modified"])
changes = to_dt(changes, ["date_created"])
perims  = to_dt(perims,  ["date_created","source_date_current"])

In [ ]:
# merging datasets

merged = changes.merge(events, left_on="geo_event_id", right_on="id", how="left", suffixes=("_chg","_evt"))
merged = merged.merge(perims, on="geo_event_id", how="left", suffixes=("","_perim"))
merged = merged.merge(evac_map[["geo_event_id","uid_v2"]].drop_duplicates(),on="geo_event_id", how="left")
merged = merged.merge(evac.drop_duplicates("uid_v2"),on="uid_v2", how="left", suffixes=("","_evac"))

print("Merged dataset:", merged.shape)

In [ ]:
merged.tail(10)

In [ ]:

# Missing values
missing_values = pd.DataFrame({
    'Variable': merged.columns,
    'Missing values count': merged.isnull().sum().values,
    'Missing values %': (merged.isnull().sum().values / len(merged) * 100)})

# Unique values
unique_values = pd.DataFrame({
    'Variable': merged.columns,
    'Unique values count': merged.nunique().values})

# Data types
feature_types = pd.DataFrame({
    'Variable': merged.columns,
    'Data type': merged.dtypes.astype(str)})

# Merge all summaries
summary_df = (missing_values
    .merge(unique_values, on='Variable', how='left')
    .merge(feature_types, on='Variable', how='left'))

# Sort by missing percentage
summary_df = summary_df.sort_values(by='Missing values %', ascending=False)

# Style and display with Matplotlib colormap
summary_df.style.background_gradient(cmap='rocket_r').format({
    'Missing values %': '{:.2f}',
    'Unique values count': '{:,}',
    'Missing values count': '{:,}'})


High Missingness ( > 90% missing )
   - evacuation_warnings (99.6%), evacuation_orders (99.3%), evacuation_notes (79.5%), status (94.5%)
     
     Interpretation: These fields are specific to a subset of incidents (fires that reached evacuation or mapping stages). They are not general attributes for all wildfires.

Moderate Missingness
   - source_acres (12.7%), region_id (10.8%), source_attribution (10.8%), geom_evac (10.8%)

   Interpretation: Fires without mapped perimeters or region-level attribution may correspond to smaller, short-lived, or unreported incidents.

Low Missingness (<5% missing)
   - containment (2.2%), is_fps (2.1%), acreage (0.5%)
   
   Interpretation: These core variables are the most dependable and should form the foundation of descriptive or predictive modeling.

No Missingness (0%)
   - changes, date_created_chg → Complete logs of event updates, ideal for constructing time-series or response time analyses.

Unique Values Insights
   - geo_event_id and id have >40K unique values → many distinct wildfire events.
   - acreage, containment, and source_acres show meaningful numeric variability
   - categorical fields (e.g., approval_status, is_fps, is_prescribed) have few unique values

## Approach

In [ ]:
# alert lag ( date_modified (in events)- date_created_chg (in changelog))

ref_col = "date_modified" if "date_modified" in merged.columns else "date_created_evt"
if ref_col not in merged.columns and "date_created" in merged.columns:
    merged.rename(columns={"date_created":"date_created_evt"}, inplace=True)
    ref_col = "date_created_evt"

merged["lag_min"] = (merged[ref_col] - merged["date_created_chg"]).dt.total_seconds() / 60
merged["lag_min"] = merged["lag_min"].replace([np.inf, -np.inf], np.nan)
merged["delayed"] = (merged["lag_min"] > 1440).astype(int) #1440 mins = 24h

print("Delayed rate:", round(merged["delayed"].mean(), 4))

Using a threshold of 24 hours (1440 minutes), 94.6% of alerts are considered delayed.
   - This high proportion suggests:
   event metadata and changelogs are often updated well after initial detection. Communication or reporting delays may exist between on-site activity and the official data pipeline.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
import numpy as np

main_color = "#c20235"
sns.set_theme(style="whitegrid")

print("Descriptive statistics for lag:")
print(merged['lag_min'].describe(percentiles=[.1, .25, .5, .75, .9]))

# Overall lag distribution
plt.figure(figsize=(10,5))
sns.histplot(merged['lag_min'], bins=80, kde=True, color=main_color)
plt.title("\nDistribution of Alert Lag (minutes)", fontsize=13)
plt.xlabel("Lag between change and last modification (min)")
plt.ylabel("Count")
plt.grid(True, alpha=0.3)
plt.show()


   - `lag_min` represents the time difference (in minutes) between when a change
     was recorded (`date_created_chg` from changelog) and when the corresponding
     event was last modified (`date_modified` from events).
   - A larger lag indicates slower updates or longer alert propagation.

The distribution is heavily right-skewed — most alerts occur within a few weeks, but some outliers experience extremely long update delays (hundreds of days).
  - The histogram shows a steep drop after 100,000 minutes (~70 days).
   - Most events cluster between 5,000 and 40,000 minutes (4 to 28 days).
   - The long tail indicates that a subset of incidents are updated long after the original report, possibly due to archival or post-event data corrections.

In [ ]:
# Lag by event type
plt.figure(figsize=(8,4))
sns.boxplot(
    data=merged,
    x='geo_event_type',
    y='lag_min',
    showfliers=False,
    color=main_color,
    width=0.6
)

plt.title("\nAlert Lag by Event Type", fontsize=12, fontweight="bold")
plt.ylabel("Lag (minutes)")
plt.xlabel("")
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

Wildfires show very large alert delays (median ~17 days), with wide variation in update times.
Location events have near-zero lag, meaning they are updated almost immediately.

In [ ]:
import matplotlib.pyplot as plt

if "status" in merged.columns:
    avg_lag_by_status = (
        merged.groupby("status", dropna=False)["lag_min"]
        .mean()
        .sort_values()
    )

    plt.figure(figsize=(8,4))
    avg_lag_by_status.plot(
        kind="barh",
        color= main_color
    )
    plt.title("Average Alert Lag by Evacuation Status", fontsize=13)
    plt.xlabel("Mean Lag (minutes)")
    plt.ylabel("Evacuation Status")
    plt.grid(True, axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()

    print(avg_lag_by_status)



Alert lag shortens as evacuation severity increases —
orders are updated fastest, while advisories and unspecified cases lag most.
This suggests quicker communication once formal evacuations are declared.

In [ ]:
# Delay rate by fire size
if "acreage" in merged.columns:
    merged['log_acreage'] = np.log1p(pd.to_numeric(merged['acreage'], errors='coerce'))
    plt.figure(figsize=(8,5))
    sns.scatterplot(
        data=merged,
        x='log_acreage',
        y='lag_min',
        alpha=0.3,
        color=main_color
    )
    plt.title("Fire Size (log acres) vs. Alert Lag")
    plt.xlabel("Log(Acreage + 1)")
    plt.ylabel("Lag (min)")
    plt.grid(True, alpha=0.3)
    plt.show()

Larger fires tend to have shorter alert lags, though variability remains high.  
Smaller incidents show wider delays, suggesting limited reporting priority or slower detection.

In [ ]:
import folium
import numpy as np

if {"lat","lng","lag_min"}.issubset(merged.columns):
    # Sample at most 2000 points for speed
    geo_delay = merged.dropna(subset=['lat','lng','lag_min']).sample(n=min(2000, len(merged)), random_state=42)
    geo_delay['Lag (min)'] = geo_delay['lag_min'].round(1)
    geo_delay['Popup'] = (
        "<b>" + geo_delay['name'].astype(str) + "</b><br>"
        "Lag: " + geo_delay['Lag (min)'].astype(str) + " min<br>"
        "Containment: " + geo_delay['containment'].astype(str) + "%<br>"
        "Acreage: " + geo_delay['acreage'].astype(str)
    )

    m = folium.Map(
        location=[geo_delay['lat'].mean(), geo_delay['lng'].mean()],
        zoom_start=6,
        tiles="CartoDB positron"
    )

    # markers
    for _, row in geo_delay.iterrows():
        color = (
            "#67a56f" if row["lag_min"] < 1440 else  # <1 day
            "#E26A4A" if row["lag_min"] < 10080 else  # <1 week
            "#4B0082"  # >1 week
        )
        folium.CircleMarker(
            location=[row['lat'], row['lng']],
            radius=4,
            color=color,
            fill=True,
            fill_opacity=0.7,
            popup=folium.Popup(row['Popup'], max_width=300)
        ).add_to(m)

    # legend
    legend_html = """
     <div style="
         position: fixed;
         bottom: 50px; left: 50px; width: 180px; height: 120px;
         background-color: white;
         border:2px solid grey; z-index:9999; font-size:14px;
         box-shadow: 2px 2px 4px rgba(0,0,0,0.3);
         ">
         &nbsp;<b>Alert Lag Legend</b><br>
         &nbsp;<i style="background:#67a56f;color:#67a56f;">.....</i>&nbsp; < 1 day<br>
         &nbsp;<i style="background:#E26A4A;color:#E26A4A;">.....</i>&nbsp; 1–7 days<br>
         &nbsp;<i style="background:#4B0082;color:#4B0082;">.....</i>&nbsp; > 7 days<br>
     </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))

    m.save("delay_map_sampled_with_legend.html")
    display(m)


- Most incidents are in the western U.S.
- Long alert lags (>7 days) dominate, especially across CA/OR/WA and the Intermountain West.


  ML models

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, roc_curve
)
import shap
import joblib

sns.set_theme(style="whitegrid")


In [ ]:
# data preparation
features = ["containment", "acreage", "is_fps", "is_prescribed", "lat", "lng"]
merged["delayed"] = (merged["lag_min"] > 60).astype(int)

for f in features:
    merged[f] = pd.to_numeric(merged[f], errors="coerce")

model_data = merged.dropna(subset=features + ["delayed"]).copy()
X = model_data[features]
y = model_data["delayed"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

In [ ]:
# # random forest
# rf_params = {
#     "n_estimators": [200, 400],
#     "max_depth": [10, 20, None],
#     "min_samples_split": [2, 5],
#     "min_samples_leaf": [1, 2],
#     "max_features": ["sqrt", "log2"]
# }

# rf = RandomForestClassifier(random_state=42, class_weight="balanced", n_jobs=-1)
# rf_grid = GridSearchCV(rf, rf_params, scoring="roc_auc", cv=3, n_jobs=-1, verbose=2)
# rf_grid.fit(X_train_scaled, y_train)

# best_rf = rf_grid.best_estimator_
# print("\nBest RF params:", rf_grid.best_params_)

# y_pred_rf = best_rf.predict(X_test_scaled)
# y_prob_rf = best_rf.predict_proba(X_test_scaled)[:, 1]

# print("\n Tuned Random Forest Performance")
# print(classification_report(y_test, y_pred_rf, digits=3))
# print("ROC-AUC:", round(roc_auc_score(y_test, y_prob_rf), 4))

In [ ]:
# # gradient boosting
# gb_params = {
#     "n_estimators": [100, 200],
#     "learning_rate": [0.05, 0.1],
#     "max_depth": [3, 5],
#     "min_samples_split": [2, 5],
#     "min_samples_leaf": [1, 2],
#     "subsample": [0.8, 1.0]
# }

# gb = GradientBoostingClassifier(random_state=42)
# gb_grid = GridSearchCV(gb, gb_params, scoring="roc_auc", cv=3, n_jobs=-1, verbose=2)
# gb_grid.fit(X_train_scaled, y_train)

# best_gb = gb_grid.best_estimator_
# print("\nBest GB params:", gb_grid.best_params_)

# y_pred_gb = best_gb.predict(X_test_scaled)
# y_prob_gb = best_gb.predict_proba(X_test_scaled)[:, 1]

# print("\n Tuned Gradient Boosting Performance")
# print(classification_report(y_test, y_pred_gb, digits=3))
# print("ROC-AUC:", round(roc_auc_score(y_test, y_prob_gb), 4))

In [ ]:
# # model comparison
# comparison = pd.DataFrame({
#     "Model": ["Random Forest (tuned)", "Gradient Boosting (tuned)"],
#     "ROC_AUC": [
#         roc_auc_score(y_test, y_prob_rf),
#         roc_auc_score(y_test, y_prob_gb)
#     ]
# }).sort_values("ROC_AUC", ascending=False)

# print("\nModel Comparison Summary:")
# print(comparison)

# plt.figure(figsize=(7,4))
# sns.barplot(x="ROC_AUC", y="Model", data=comparison, palette="YlOrBr")
# plt.title("Model Comparison — Tuned Models")
# plt.xlabel("AUC Score")
# plt.ylabel("")
# plt.xlim(0.5, 1)
# plt.tight_layout()
# plt.show()

In [ ]:
# # SHAP explainability - best model
# best_model = best_rf if roc_auc_score(y_test, y_prob_rf) >= roc_auc_score(y_test, y_prob_gb) else best_gb
# model_name = "Random Forest" if best_model == best_rf else "Gradient Boosting"

# print(f"\nBest overall model: {model_name}")

# explainer = shap.TreeExplainer(best_model)
# shap_values = explainer.shap_values(X_test_scaled)

# if isinstance(shap_values, list):
#     shap_values = shap_values[1]

# shap.summary_plot(shap_values, X_test, plot_type="bar", show=False)
# plt.title(f"SHAP Summary — {model_name}")
# plt.tight_layout()
# plt.show()

# shap.summary_plot(shap_values, X_test, show=False)
# plt.title(f"SHAP Impact Plot — {model_name}")
# plt.tight_layout()
# plt.show()

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score, roc_curve)

features = ["containment", "acreage", "is_fps", "is_prescribed", "lat", "lng"]
merged["delayed"] = (merged["lag_min"] > 60).astype(int)

for f in features:
    merged[f] = pd.to_numeric(merged[f], errors="coerce")

model_data = merged.dropna(subset=features + ["delayed"]).copy()
X = model_data[features]
y = model_data["delayed"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train size: {X_train.shape}, Test size: {X_test.shape}")

# SCALING

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# BASELINE: RandomForest

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced"
)
rf.fit(X_train_scaled, y_train)

y_pred = rf.predict(X_test_scaled)
y_prob = rf.predict_proba(X_test_scaled)[:, 1]

print("\n=== Random Forest Performance ===")
print(classification_report(y_test, y_pred, digits=3))
print("ROC-AUC:", round(roc_auc_score(y_test, y_prob), 4))

# FEATURE IMPORTANCE

importances = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=False)
plt.figure(figsize=(7,4))
sns.barplot(x=importances.values, y=importances.index, palette="YlOrBr")
plt.title("Feature Importance — Random Forest")
plt.xlabel("Importance Score")
plt.tight_layout()
plt.show()

# CONFUSION MATRIX

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# ROC CURVE

fpr, tpr, _ = roc_curve(y_test, y_prob)
plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, color="#c20235", lw=2, label=f"AUC = {roc_auc_score(y_test, y_prob):.3f}")
plt.plot([0,1],[0,1],'--',color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve – Random Forest")
plt.legend()
plt.tight_layout()
plt.show()

Random Forest results:
- Top feature: acreage - larger fires strongly predict delayed alerts.
- Model AUC = 0.78 - moderate discrimination between delayed vs. timely alerts.
- Recall(1) = 0.63, Precision(1) = 0.99 - model detects most delayed cases but rarely flags timely ones.

In [ ]:
# Gradient Boosting

gb = GradientBoostingClassifier(random_state=42)
gb.fit(X_train_scaled, y_train)

y_pred_gb = gb.predict(X_test_scaled)
y_prob_gb = gb.predict_proba(X_test_scaled)[:,1]

print("\n= Gradient Boosting Performance =")
print(classification_report(y_test, y_pred_gb, digits=3))
print("ROC-AUC:", round(roc_auc_score(y_test, y_prob_gb), 4))

# MODEL COMPARISON

plt.figure(figsize=(6,5))
plt.plot(*roc_curve(y_test, y_prob)[:2], label=f"RandomForest (AUC={roc_auc_score(y_test, y_prob):.3f})")
plt.plot(*roc_curve(y_test, y_prob_gb)[:2], label=f"GradBoost (AUC={roc_auc_score(y_test, y_prob_gb):.3f})")
plt.plot([0,1],[0,1],'--',color="gray")
plt.title("ROC Comparison: Random Forest vs Gradient Boosting")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.tight_layout()
plt.show()

Gradient Boosting outperforms Random Forest (AUC 0.84 vs. 0.78),
showing stronger ability to detect delayed alerts.
It achieves near-perfect recall for delayed cases, though at the cost of slightly lower precision.


In [ ]:
import shap
shap.initjs()

explainer_rf = shap.TreeExplainer(rf)
shap_values_rf = explainer_rf.shap_values(X_test_scaled)
if isinstance(shap_values_rf, list):
    shap_values_rf = shap_values_rf[1]

print("\nTop SHAP features for Random Forest:")
shap.summary_plot(shap_values_rf, X_test, plot_type="bar", color="#E26A4A")
shap.summary_plot(shap_values_rf, X_test, color="#E26A4A")
shap.dependence_plot("acreage", shap_values_rf, X_test, display_features=X_test)

explainer_gb = shap.Explainer(gb)
shap_values_gb = explainer_gb(X_test_scaled)

print("\nTop SHAP features for Gradient Boosting:")
shap.summary_plot(shap_values_gb, X_test, plot_type="bar", color="#c20235")
shap.summary_plot(shap_values_gb, X_test, color="#c20235")
shap.dependence_plot("containment", shap_values_gb.values, X_test, display_features=X_test)

## Results

Report your final results and key insights:
- Metrics: Precision, Recall, ROC AUC, RMSE, etc.
- Key findings or visualizations (include in slides)
- Any limitations or ethical considerations

## Team Contributions

| Name         | Contributions                                |
|--------------|----------------------------------------------|
| Gianina Petrașcu       | Feature engineering, model tuning            |
| Ioana Bîrlan           | EDA, preprocessing, geospatial joins         |